In [2]:
import pandas as pd
import statsmodels.api as sm
import scipy
import numpy as np

Я взял dataset https://www.kaggle.com/uciml/iris, в котором содержится описание трёх видов ириса

In [3]:
df = pd.read_csv('iris.csv')

In [4]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
df.Species.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

Хочу взять два вида цветов, найти среднюю длину чашелистика каждого вида, посчитать количество цветков длиннее среднего размера и сравнить две группы

In [6]:
count_vir = df[df.Species == 'Iris-virginica'].shape[0]
count_ver = df[df.Species == 'Iris-versicolor'].shape[0]

count_vir, count_ver

(50, 50)

In [7]:
mean_vir = df[df.Species == 'Iris-virginica'].SepalLengthCm.mean()
mean_ver = df[df.Species == 'Iris-versicolor'].SepalLengthCm.mean()

mean_vir, mean_ver

(6.587999999999998, 5.936)

In [8]:
cond_vir = (df.Species == 'Iris-virginica') & (df.SepalLengthCm > mean_vir)
part_vir = df[cond_vir].shape[0]

cond_ver = (df.Species == 'Iris-versicolor') & (df.SepalLengthCm > mean_ver)
part_ver = df[cond_ver].shape[0]

part_vir, part_ver

(22, 24)

In [9]:
p1 = part_vir / count_vir
p2 = part_ver / count_ver
p_wave = (part_vir + part_ver) / (count_vir + count_ver)

p1, p2, p_wave

(0.44, 0.48, 0.46)

In [10]:
z = (p1 - p2) / (p_wave * (1 - p_wave) * (1 / count_ver + 1 / count_vir)) ** 0.5

z

-0.4012861769525638

#### Двусторонняя гипотеза

Нулевая гипотеза (H0) состоит в том, что пропорции одинаковы;

Альтернативная гипотеза (H1) состоит в том, что пропорции разные.

In [11]:
p_value = 2 * scipy.stats.norm.sf(abs(z))

p_value

0.6882094395496218

In [12]:
count = np.array([part_vir, part_ver])
nobs = np.array([count_vir, count_ver])

sm.stats.proportions_ztest(count, nobs, alternative = 'two-sided')

(-0.4012861769525638, 0.6882094395496218)

In [13]:
if p_value > 0.05:
    print('Не можем отвергнуть гипотезу H0', '\nZ =', z, '\np_value =', p_value)
else:
    print('Есть основания отвергнуть гипотезу H0', '\nZ =', z, '\np_value =', p_value)

Не можем отвергнуть гипотезу H0 
Z = -0.4012861769525638 
p_value = 0.6882094395496218


#### Левосторонняя/правосторонняя гипотеза

1) H0: π1 < π2, HA: π1 >= π2;

2) H0: π1 > π2, HA: π1 <= π2.

In [14]:
p_value_left = scipy.stats.norm.sf(abs(z))
p_value_right = 1 - scipy.stats.norm.sf(abs(z))

p_value_left, p_value_right

(0.3441047197748109, 0.6558952802251892)

In [15]:
left = sm.stats.proportions_ztest(count, nobs, alternative = 'smaller')
right = sm.stats.proportions_ztest(count, nobs, alternative = 'larger')

left, right

((-0.4012861769525638, 0.3441047197748109),
 (-0.4012861769525638, 0.6558952802251892))

#### Левосторонняя гипотеза

In [16]:
if p_value_left < 0.05:
    print('Не можем отвергнуть нулевую гипотезу о том, что π1 < π2', '\nZ =', z, '\np_value =', p_value_left)
else:
    print('Не можем принять нулевую гипотезу о том, что π1 < π2', '\nZ =', z, '\np_value = ', p_value_left)

Не можем принять нулевую гипотезу о том, что π1 < π2 
Z = -0.4012861769525638 
p_value =  0.3441047197748109


#### Правосторонняя гипотеза

In [18]:
if p_value_right > 0.95:
    print('Не можем отвергнуть нулевую гипотезу о том, что π1 ≥ π2', '\nZ =', z, '\np_value =', p_value_right)
else:
    print('Не можем принять нулевую гипотезу о том, что π1 ≥ π2', '\nZ =', z, '\np_value =', p_value_right)

Не можем принять нулевую гипотезу о том, что π1 ≥ π2 
Z = -0.4012861769525638 
p_value = 0.6558952802251892
